# Linear regression from scratch
## Pytorch and Autograd basics

This notebook is going to go through a first principals approach to creating a linear regression model using Pytorch and its built in autodif library, Autograd. We are only going to need two libraries for this, Numpy and Torch. Numpy is specifically just for creating tensors in familiar ways.

In [40]:
import numpy as np
import torch

## Tensors, N dimensional arrays and linear equations. 

The illustration below highlights this really nicely, where by there is increased dimensionality from a vector, a 1-D structure, to a matrix, a 2-D structure and tensors, which are N-Dimensional and can also be interpreted as stacked matrices.

<img src="scalar-vector-matrix-tensor.jpg">

The first thing we are going to do is instantiate some tensors with the values to solve the linear equation below:

$y = W * X + B$

In [41]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3.)
print(x)
print(w)
print(b)

tensor(1., requires_grad=True)
tensor(2., requires_grad=True)
tensor(3.)


We have now created the W X and B variables, and declared ```requires_grad=True``` for X and W. This creates a computational graph for W and X which tracks the operation history for these varialbes and forms a backward graph for gradient calculation.

<img src="grad_graph.png">


In [42]:
# Arithmetic operations
y = w * x + b
print(y)

tensor(5., grad_fn=<AddBackward0>)


So far all we have done is solve the linear combination of W, X and B for y which is 5. Now we can illustrate what we are using the autodiff capabilities of pytorch/autograd for. Given that we know the product of these tensors is 5, we know that the product of W or X and it's gradient, the slope of the equation at that point, plus the intercept B, will give us 5. We can prove this below. 

In [43]:
y.backward()
print(x.grad)
print(w.grad)
print(b)

tensor(2.)
tensor(1.)
tensor(3.)


To check: x = 1, it's gradient is 2 so X * X.grad + B = 5. Keep this in mind, this will come in handy later, when we are using this feature to minimize a loss function to converge a very simple linear regression model, and because we are still in lockdown we are going to try and fit the flavour (subjective) and degree of crustiness (also subjective) of my home made sour dough to the measured proofing time, acidity and age of my starter.

## Linear Regression Data

In [44]:
# Input (Proofing time, acidity, age of starter)
inputs = np.array([[12, 4.5, 32], 
                   [15, 4.2, 40], 
                   [18, 4.2, 45], 
                   [24, 4.0, 51], 
                   [30, 3.8, 58],
                   [18, 3.8, 65],
                   [15, 4.0, 70],
                   [15, 4.1, 78],
                   [18, 3.9, 81],
                   [24, 4.0, 90]], dtype='float32')
# Targets (Flavour, Crust)
targets = np.array([[70, 70], 
                    [72, 75], 
                    [80, 70], 
                    [85, 75], 
                    [85, 80],
                    [82, 85], 
                    [90, 85], 
                    [87, 90], 
                    [85, 90], 
                    [90, 95]], dtype='float32')
print(inputs) # This is an Array

[[12.   4.5 32. ]
 [15.   4.2 40. ]
 [18.   4.2 45. ]
 [24.   4.  51. ]
 [30.   3.8 58. ]
 [18.   3.8 65. ]
 [15.   4.  70. ]
 [15.   4.1 78. ]
 [18.   3.9 81. ]
 [24.   4.  90. ]]


In [45]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs) # we are using the from_numpy method which doesn't copy the data but provides a reference.
targets = torch.from_numpy(targets) # This is an efficient method when the data is large, probably unnecessary for data of this size.
print(inputs) # This is the converted Tensor where the type has been converted to float32.

tensor([[12.0000,  4.5000, 32.0000],
        [15.0000,  4.2000, 40.0000],
        [18.0000,  4.2000, 45.0000],
        [24.0000,  4.0000, 51.0000],
        [30.0000,  3.8000, 58.0000],
        [18.0000,  3.8000, 65.0000],
        [15.0000,  4.0000, 70.0000],
        [15.0000,  4.1000, 78.0000],
        [18.0000,  3.9000, 81.0000],
        [24.0000,  4.0000, 90.0000]])


### So we now have a matrix of 10x3 inputs, and a matrix of 10x2 targets.

In an ideal situation, we would multiply our inputs, X, by some weights w, and add a bias, b. This will return our target values of y for the correct values of w and b. Now, we are unlikely to guess the correct values of w and b on the first guess, but there are a few ways forward. We could grid or random search a matrix of values to brute force this, but this is not a very sophisticated method and will not necessarily find the minima for latent space if that area is outside the defined search area. 

$$\begin{bmatrix} y1 & y1 \\ y2 & y2 \\ y3 & y3 \end{bmatrix} = \begin{bmatrix} x1 & x1 & x1 \\ x2 & x2 & x2 \\ x3 & x3 & x3 \end{bmatrix}\begin{bmatrix} w1 & w1 \\ w2 & w2 \\ w3 & w3 \end{bmatrix} + \begin{bmatrix} b1 & b1 \\ b2 & b2 \\ b3 & b3 \end{bmatrix}$$

So we will first create random weights and biases, requiring grad for autodifferentiation. The weights are in the shape 2x3, we will multiply the inputs by the transpose of this matrix to obtain the desired shape of the target tensor.

In [46]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.5543, -0.6560,  1.4981],
        [ 0.0123,  1.5953, -0.6069]], requires_grad=True)
tensor([-0.9139,  0.7198], requires_grad=True)


### This model is just the formalisation of the y = mx + b above, with the matrix multiplication. 

In [47]:
# Define the model
def model(x):
    return x @ w.t() + b

### Below is a single example of how this equation solves for y for a single, random, set of weights and biases.

In [48]:
inputs @ w.t() + b

tensor([[ 50.7237, -11.3739],
        [ 64.5679, -16.6706],
        [ 73.7212, -19.6681],
        [ 86.1666, -23.5547],
        [100.1101, -28.0481],
        [103.9450, -32.4437],
        [109.6413, -35.1959],
        [121.5603, -39.8913],
        [127.8486, -41.9941],
        [144.5914, -47.2227]], grad_fn=<AddBackward0>)

In [49]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 50.7237, -11.3739],
        [ 64.5679, -16.6706],
        [ 73.7212, -19.6681],
        [ 86.1666, -23.5547],
        [100.1101, -28.0481],
        [103.9450, -32.4437],
        [109.6413, -35.1959],
        [121.5603, -39.8913],
        [127.8486, -41.9941],
        [144.5914, -47.2227]], grad_fn=<AddBackward0>)


In [50]:
# Compare with targets
print(targets)

tensor([[70., 70.],
        [72., 75.],
        [80., 70.],
        [85., 75.],
        [85., 80.],
        [82., 85.],
        [90., 85.],
        [87., 90.],
        [85., 90.],
        [90., 95.]])


### The above two cells show the calculated predictions for y and the targets we are aiming for. They are not very close, and it is pretty easy to calculate how close! 

To do that we will use mean square error, MSE, which finds the average squared error between two tensors. In this case, the difference between the predicted values and the target values is squared, the tensor summed, and then divided by the number of elements in the tensor. 

In [51]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [52]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(6739.4248, grad_fn=<DivBackward0>)


### Here we are now going to use the features of autograd on the loss product.

In [53]:
# Compute gradients
loss.backward()

### For the components of the computational graph that make up loss that require autograd in their instantiation calling backward on the object loss will automatically calculate the gradient for these components, in this case, w and b.

In [54]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.5543, -0.6560,  1.4981],
        [ 0.0123,  1.5953, -0.6069]], requires_grad=True)
tensor([[  331.4986,    60.6884,  1362.0184],
        [-2122.6865,  -447.5899, -7126.9585]])


In [55]:
# Gradients for bias
print(b)
print(b.grad)

tensor([-0.9139,  0.7198], requires_grad=True)
tensor([  15.6876, -111.1063])


### Now what the gradient tells us, in practical terms, is in which direction the nearest function minima is. If the gradient is positive, reducing the value will step the function back toward the minima, if the gradient is negative making it larger, will bring it closer to that minima. So if we adjust the weights and biases in the correct direction, for every step in a training loop we will coninually move the values off the weights and biases toward the optimal values, and reduce the loss as much as we can.

In [56]:
# Train for 100 epochs
for i in range(100000): # do this 100,000 times
    preds = model(inputs) # predict the targets for the input values
    loss = mse(preds, targets) # calculate the loss
    loss.backward() # calculate the gradients
    with torch.no_grad(): # the weight update step shouldn't be added to the computational graph.
        w -= w.grad * 1e-5 # incriment the weights by a tiny percentage of the gradient
        b -= b.grad * 1e-5 # incriment the bias by a tiny percentage of the gradient
        w.grad.zero_() # clear the gradients
        b.grad.zero_() # clear the gradients

The final weights and biases for this model are:

In [57]:
print(w)
print(b)

tensor([[1.0813, 7.9036, 0.4634],
        [0.4770, 8.0436, 0.6078]], requires_grad=True)
tensor([0.9097, 2.1006], requires_grad=True)


With a final loss of:

In [58]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(23.5815, grad_fn=<DivBackward0>)


### Now let's compare the predicted final values with the actual targets to see how well we have model the fit.

In [59]:
# Print predictions
preds

tensor([[ 64.2809,  63.4700],
        [ 68.8611,  67.3503],
        [ 74.4221,  71.8202],
        [ 82.1099,  76.7202],
        [ 90.2610,  82.2280],
        [ 80.5289,  80.7585],
        [ 81.1827,  83.9751],
        [ 85.6803,  89.6418],
        [ 88.7338,  91.2874],
        [100.1829, 100.4239]], grad_fn=<AddBackward0>)

In [60]:
# Print targets
targets

tensor([[70., 70.],
        [72., 75.],
        [80., 70.],
        [85., 75.],
        [85., 80.],
        [82., 85.],
        [90., 85.],
        [87., 90.],
        [85., 90.],
        [90., 95.]])

### Lastly! It is 100% possible to easily commute this to a GPU by moving the tensors and model to a gpu by calling device and pointing to the required bit of hardware

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inputs = torch.from_numpy(inputs).to(device)
targets = torch.from_numpy(targets).to(device)

# Train for 100 epochs
for i in range(100):
    preds = model(inputs).to(device)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

# Using the NN built in methods of pytorch...

In [211]:
import torch
import torch.nn as nn

# Create class
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

input_dim = 3
output_dim = 2

model = LinearRegressionModel(input_dim, output_dim)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model.to(device)

criterion = nn.MSELoss()

learning_rate = 0.0001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10000

for epoch in range(epochs):
    epoch += 1
    # Convert numpy array to torch Variable
    values = inputs.clone().detach().requires_grad_(True)
    labels = targets.clone().detach().requires_grad_(True)
    #inputs = inputs.clone().detach().to(device)
    #labels = targets.clone().detach().to(device)

    # Clear gradients w.r.t. parameters
    optimizer.zero_grad() 

    # Forward to get output
    outputs = model(values)

    # Calculate Loss
    loss = criterion(outputs, labels)

    # Getting gradients w.r.t. parameters
    loss.backward()

    # Updating parameters
    optimizer.step()
    
    if epoch % 100 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 100, loss 231.43313598632812
epoch 200, loss 192.86708068847656
epoch 300, loss 173.49008178710938
epoch 400, loss 162.86288452148438
epoch 500, loss 156.2476348876953
epoch 600, loss 151.49337768554688
epoch 700, loss 147.62332153320312
epoch 800, loss 144.19320678710938
epoch 900, loss 141.00059509277344
epoch 1000, loss 137.9528045654297
epoch 1100, loss 135.00689697265625
epoch 1200, loss 132.14260864257812
epoch 1300, loss 129.34991455078125
epoch 1400, loss 126.62345886230469
epoch 1500, loss 123.96012878417969
epoch 1600, loss 121.35768127441406
epoch 1700, loss 118.81441497802734
epoch 1800, loss 116.328857421875
epoch 1900, loss 113.89957427978516
epoch 2000, loss 111.52531433105469
epoch 2100, loss 109.20477294921875
epoch 2200, loss 106.93680572509766
epoch 2300, loss 104.7201156616211
epoch 2400, loss 102.55360412597656
epoch 2500, loss 100.43608093261719
epoch 2600, loss 98.36651611328125
epoch 2700, loss 96.34379577636719
epoch 2800, loss 94.36680603027344
epoch 290